In [49]:
from casatools import simulator, image, table, coordsys, measures, componentlist, quanta, ctsys
from casatasks import tclean, ft, imhead, listobs, exportfits, flagdata, bandpass, applycal
from casatasks.private import simutil
from cngi.conversion import convert_ms, convert_image

import os
import pylab as pl
import numpy as np
#from astropy.io import fits
#from astropy.wcs import WCS

from casatools import simulator
from casatasks import tclean

In [50]:
sm = simulator()
ia = image()
tb = table()
cs = coordsys()
me = measures()
qa = quanta()
cl = componentlist()
mysu = simutil.simutil()

In [51]:
dovp = True
#gridder = 'standard'
gridder = 'mosaic'
#ms_name = 'zenith_point_source_sim_dovp_' + str(dovp) + '.ms'
#im_name = 'zenith_point_source_sim_img/point_source_sim_dovp_' + str(dovp) + '_gridder_' + gridder

ms_name = 'point_source_sim_vis/mosaic_point_source_sim_dovp_' + str(dovp) + '.ms'
im_name = 'point_source_sim_img/mosaic_point_source_sim_dovp_' + str(dovp) + '_gridder_' + gridder

#conf_file = 'tel_config/vla_small.d.cfg' #Small array with 3 antennae
conf_file = 'tel_config/vla.d.cfg'
#time_interval = ['-5.0h','5.0h']
#integration_time = '600.0s'
time_interval = ['-5.0h','0.0h']
integration_time = '3600.0s'
time_interval2 = ['0.0h','5.0h']
integration_time = '3600.0s' #Can be reduced to increase resolution

In [52]:
os.system('rm -rf ' + ms_name)

0

In [53]:
sm.open(ms=ms_name);

In [54]:
q = mysu.readantenna(conf_file)

In [55]:
#CASA5
#(x,y,z,d,an,an2,telname, obspos) = mysu.readantenna(conf_file)

#CASA6
(x,y,z,d,an,an2,telname, obspos) = mysu.readantenna(conf_file)

In [56]:
## Set the antenna configuration
sm.setconfig(telescopename=telname,
                        x=x,
                        y=y,
                        z=z,
                        dishdiameter=d,
                        mount=['alt-az'],
                        antname=an,
                        coordsystem='local',
                        referencelocation=me.observatory(telname));

In [57]:
## Set the polarization mode (this goes to the FEED subtable)
sm.setfeed(mode='perfect R L', pol=['']);

In [58]:
'''
sm.setspwindow(spwname="SBand",
               freq='3.0GHz',
               deltafreq='0.4GHz',
               freqresolution='0.01GHz',
               nchannels=3,
               refcode='LSRK',
               stokes='RR RL LR LL');
'''

sm.setspwindow(spwname="SBand",
               freq='3.0GHz',
               deltafreq='0.4GHz',
               freqresolution='0.01GHz',
               nchannels=3,
               refcode='LSRK',
               stokes='RR LL');

In [59]:
sm.setfield( sourcename="fake",sourcedirection=me.direction(rf='J2000', v0='20h00m28.5s',v1='+40d44m01.5s'))
sm.setfield( sourcename="fake2",sourcedirection=me.direction(rf='J2000', v0='19h58m28.5s',v1='+40d44m01.5s'))
#sm.setfield( sourcename="fake",sourcedirection=me.direction(rf='J2000', v0='0h0m0.0s',v1='+90d00m0.0s')) #Zenith

True

In [60]:
## Leave autocorrelations out of the MS.
sm.setauto(autocorrwt=0.0)

True

In [61]:
sm.settimes(integrationtime=integration_time,
                 usehourangle=True,
                 referencetime=me.epoch('UTC','2019/10/4/00:00:00'));

In [62]:
sm.observe(sourcename="fake",
               spwname='SBand',
               starttime=time_interval[0],
               stoptime=time_interval[1]);
sm.observe(sourcename="fake2",
               spwname='SBand',
               starttime=time_interval2[0],
               stoptime=time_interval2[1]); #Need to make this time different

In [63]:
## Close the simulator
sm.close()

True

In [64]:
clname=ms_name.split('.')[0] + '.cl'
os.system('rm -rf '+ clname)

0

In [65]:
# Add sources, one at a time.
# Call multiple times to add multiple sources. ( Change the 'dir', obviously )
#    cl.addcomponent(dir='J2000 19h59m28.5s +40d44m01.5s',
#                        flux=5.42,            # For a gaussian, this is the integrated area.
#                        fluxunit='Jy',
#                        freq='LSRK 3.0GHz',
#                        shape='point',       ## Point source
#    #                    shape='gaussian',   ## Gaussian
#    #                    majoraxis="5.0arcmin",
#    #                    minoraxis='2.0arcmin',
#                        spectrumtype="constant",
#                        index=-1.0)
#Sim
cl.addcomponent(dir='J2000 20h00m28.5s +40d44m01.5s',
                    flux=2.17,            # For a gaussian, this is the integrated area.
                    fluxunit='Jy',
                    freq='LSRK 3.0GHz',
                    shape='point',       ## Point source
#                    shape='gaussian',   ## Gaussian
#                    majoraxis="5.0arcmin",
#                    minoraxis='2.0arcmin',
                    spectrumtype="constant",
                    index=-1.0)

cl.addcomponent(dir='J2000 19h58m28.5s +40d44m01.5s',
                    flux=2.17,            # For a gaussian, this is the integrated area.
                    fluxunit='Jy',
                    freq='LSRK 3.0GHz',
                    shape='point',       ## Point source
#                    shape='gaussian',   ## Gaussian
#                    majoraxis="5.0arcmin",
#                    minoraxis='2.0arcmin',
                    spectrumtype="constant",
                    index=-1.0)
#    #Zenith
#    cl.addcomponent(dir='J2000 19h59m0.0s +89d54m01.5s',
#                        flux=2.17,            # For a gaussian, this is the integrated area.
#                        fluxunit='Jy',
#                        freq='LSRK 3.0GHz',
#                        shape='point',       ## Point source
#    #                    shape='gaussian',   ## Gaussian
#    #                    majoraxis="5.0arcmin",
#    #                    minoraxis='2.0arcmin',
#                        spectrumtype="constant",
#                        index=-1.0)


#    cl.addcomponent(dir='J2000 19h59m0.0s +40d51m01.5s',
#                        flux=3.145,            # For a gaussian, this is the integrated area.
#                        fluxunit='Jy',
#                        freq='LSRK 3.0GHz',
#                        shape='point',       ## Point source
#    #                    shape='gaussian',   ## Gaussian
#    #                    majoraxis="5.0arcmin",
#    #                    minoraxis='2.0arcmin',
#                        spectrumtype="constant",
#                        index=-1.0)

#    cl.addcomponent(dir='J2000 19h59m55.5s +40d27m01.5s',
#                        flux=7.56,            # For a gaussian, this is the integrated area.
#                        fluxunit='Jy',
#                        freq='LSRK 3.0GHz',
#                        shape='point',       ## Point source
#    #                    shape='gaussian',   ## Gaussian
#    #                    majoraxis="5.0arcmin",
#    #                    minoraxis='2.0arcmin',
#                        spectrumtype="constant",
#                        index=-1.0)

cl.rename(filename=clname)
cl.done()

True

In [66]:
#sm.setlimits(shadowlimit=100, elevationlimit='-180.0deg')
   

sm.openfromms(ms_name)

True

In [67]:
if dovp == False:
    sm.setvp(dovp=dovp)
else:
    sm.setvp(dovp=dovp,usedefaultvp=True,dosquint=False)

In [68]:
sm.predict(complist = clname ,incremental=False)

sm.close()

True

In [69]:
flagdata(vis=ms_name,mode='unflag')

{}

In [70]:
os.system('rm -rf '+ im_name + '.*')

0

In [71]:
#tclean(vis=ms_name,imagename=im_name,imsize=[400,200],cell=[20.0,20.0],specmode='cube',niter=0,pblimit=0.0,gridder=gridder,stokes='RR', phasecenter="J2000 20h00m28.5 40d44m01.5")
tclean(vis=ms_name,imagename=im_name,imsize=[800,400],cell=[5.0,5.0],specmode='cube',niter=0,pblimit=0.2,gridder=gridder,stokes='RR',phasecenter='J2000 19h59m28.5s +40d44m01.5s')

{}

In [72]:
convert_ms(ms_name)
#convert_image(im_name)

Completed ddi 0  process time 2.63 s.ATA...SPECTRAL_WINDOW_ID...                    
Completed subtables  process time 4.07 s...                                     
                                                  


<xarray.Dataset>
Dimensions:           (antenna_ids: 27, feed_ids: 27, field_ids: 2, observation_ids: 1, polarization_ids: 1, source_ids: 2, spw_ids: 1, state_ids: 1)
Coordinates:
  * antenna_ids       (antenna_ids) int64 0 1 2 3 4 5 6 ... 20 21 22 23 24 25 26
    antennas          (antenna_ids) <U16 'W01' 'W02' 'W03' ... 'N07' 'N08' 'N09'
  * field_ids         (field_ids) int64 0 1
    fields            (field_ids) <U5 'fake' 'fake2'
  * feed_ids          (feed_ids) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  * observation_ids   (observation_ids) int64 0
    observations      (observation_ids) <U16 'CASA simulation'
  * polarization_ids  (polarization_ids) int64 0
  * source_ids        (source_ids) int64 0 1
    sources           (source_ids) <U5 'fake' 'fake2'
  * spw_ids           (spw_ids) int64 0
  * state_ids         (state_ids) int64 0
Data variables:
    *empty*
Attributes:
    xds0:             <xarray.Dataset>\nDimensions:                (baseline:...
    ANTENNA:          <xarray.Dataset>\nDimensions:        (antenna_id: 27, d...
    FEED:             <xarray.Dataset>\nDimensions:             (d0: 27, d1: ...
    FIELD:            <xarray.Dataset>\nDimensions:        (d1: 1, d2: 2, fie...
    OBSERVATION:      <xarray.Dataset>\nDimensions:         (d1: 2, observati...
    POINTING:         <xarray.Dataset>\nDimensions:      (antenna_id: 27, d2:...
    POLARIZATION:     <xarray.Dataset>\nDimensions:       (d0: 1, d1: 2, d2: ...
    SOURCE:           <xarray.Dataset>\nDimensions:             (d0: 2, d1: 2...
    SPECTRAL_WINDOW:  <xarray.Dataset>\nDimensions:             (d1: 3, spect...
    STATE:            <xarray.Dataset>\nDimensions:   (state_id: 1)\nCoordina...